In [594]:
#КВАРТИРЫ
import pandas as pd
import random
import math
import time


buyers = pd.DataFrame()

data = pd.read_csv("flats_moscow.csv")

data['параметр 0'] = data['price']
data['параметр 1'] = data['totsp']

data['параметр 1'] = round(data['параметр 1'] * 1.352)

buyers.insert(0, "параметр 0", data['параметр 0'])
buyers.insert(1, "параметр 1", data['параметр 1'])

qual_amount = buyers.shape[1] - 1
counter_number = 2

In [595]:
def change_of_buyers(fr_name, vector):
    n = fr_name.shape[0]
    counter = 0 
    for j in range (0, n):
        condition = (vector[0] > fr_name['параметр 0'][j])
        for i in range(1, qual_amount + 1):
            name = 'параметр ' + str(i)
            condition &= vector[i] < fr_name[name][j]
        if condition:
            vector[0] = fr_name.loc[j][0]
    return vector

m = buyers.shape[0]

for i in range (0, m):
    v = change_of_buyers(buyers, buyers.loc[i][:])
    buyers.replace(buyers.loc[i][:], v, inplace = True)
    
print(buyers)

      параметр 0  параметр 1
0             50        78.0
1             50        59.0
2             50        95.0
3             50        82.0
4             50       141.0
...          ...         ...
2035          50       104.0
2036          50        81.0
2037          50        81.0
2038          50       103.0
2039          50        87.0

[2040 rows x 2 columns]


In [596]:
start_time = time.time()

mp = {}

i = 0
m = buyers.shape[0]
list_number = []

new_buyers = pd.DataFrame(columns=['параметр 0', 'параметр 1'])

while i < m:
    row = buyers.loc[i][:]
    a = row[0]
    b = row[1]
    if mp.get((a, b)) == None:
        new_buyers = new_buyers.append({'параметр 0': a, 'параметр 1': b}, ignore_index=True)
        mp[(a, b)] = 1
    else:
        mp[(a, b)] += 1
    i += 1  
    
for i in range (0, len(mp)):
    amount = mp[(new_buyers.loc[i][0], new_buyers.loc[i][1])]
    list_number.append(amount)
    
new_buyers.insert(counter_number, "количество", list_number)

new_buyers = new_buyers.sort_values(by = ['параметр 0','параметр 1'], ascending = [False, True])
new_buyers = new_buyers.reset_index(drop=True)

In [597]:
prices = list(sorted(new_buyers['параметр 0'].values.tolist(), reverse = True))
prices_set = list(sorted(set(buyers['параметр 0'].values.tolist()), reverse = True))

moments_of_time = []
for j in range (1, len(prices_set) + 1):
    moment_of_time = 0
    if j < len(prices_set):
        for k in range (prices.index(prices_set[j - 1]), prices.index(prices_set[j])):
            moment_of_time += new_buyers.loc[k][2]
            moments_of_time.append(moment_of_time)
    if j == len(prices_set):  
        for k in range (prices.index(prices_set[2]), len(prices)):
            moment_of_time += new_buyers.loc[k][2]
            moments_of_time.append(moment_of_time) 
            
new_buyers.insert(counter_number + 1, "количество покупателей", moments_of_time)

In [598]:
def amount_of_buyers_1(fr_name, vector):
    flag = False
    
    for k in range (0, fr_name.shape[0]):    
        if (new_buyers['параметр 0'][k] == vector[0]) & (new_buyers['параметр 1'][k] == vector[1]):
            a_i = fr_name['количество покупателей'][k]
            flag = True
            return (a_i, flag)
    return (0, flag)

def amount_of_buyers(fr_name, vector):
    a_i_result = 0
    t_2 = 0
    t_1 = 0
    
    for m in range (0, fr_name.shape[0]): 
        if (fr_name['параметр 0'][m] >= vector[0]):
            t_2 += new_buyers.loc[m][2]
    
    while True:
        a, f = amount_of_buyers_1(fr_name, vector)
        if not f:
#             if prices_set.index(vector[0]) 
            vector[0] = prices_set[prices_set.index(vector[0]) - 1]
            continue

        a_i_result = a
        break
    
    for m in range (0, fr_name.shape[0]): 
        if (fr_name['параметр 0'][m] >= vector[0]):
            t_1 += new_buyers.loc[m][2]
    
    if (t_1 == t_2):
        t_1 = t_2 = 0
    
    amount = a_i_result + t_2 - t_1
      
    return amount


def get_profit(fr_name, vector):
    profit = vector[0]
    for i in range(1, qual_amount + 1):
        profit -= vector[i]
    profit = profit * amount_of_buyers(fr_name, vector.copy())
    return profit

790.0

In [599]:
# Реализация алгоритма заметающей прямой для одного качества
prices = list(sorted(set(new_buyers['параметр 0'].values.tolist()), reverse = True))
parametr1 = list(sorted(set(new_buyers['параметр 1'].values.tolist()), reverse = True)) 

L = {}
L_copy = {}
list2_1 = parametr1.copy()

def price_par1(fr_name, list1, list2): 
    i = 0
    global L
    global L_copy
    map_for_delete ={}
    for m in range (0, len(list1)):
        candidate = [0, 0]
        candidate[0] = list1[m]
        list_idx = fr_name[fr_name['параметр 0'] == list1[m]].index.tolist()

        for k in range (0, len(list_idx)):
            if L.get(list2[k], 1) == 1:
                L[list2[k]] = 0
                
        for j in range (0, len(list(L.keys()))):
            
            candidate[1] = list(L.keys())[j]
            if map_for_delete.get(candidate[1], False) == True:
                continue

            a = get_profit(fr_name, candidate) 
            if L.get(list(L.keys())[j], 1) == 1:                
                L[list(L.keys())[j]] = a
            elif a >= L[list(L.keys())[j]]:
                L[list(L.keys())[j]] = a
            elif a < L[list(L.keys())[j]]:
                map_for_delete[candidate[1]] = True
    return(L)


price_par1(new_buyers, prices, parametr1)
print(d)
max_val = max(L.values())
print("--- %s seconds ---" % (time.time() - start_time))
print(max_val)

{260.0: 195.0}
--- 0.7379169464111328 seconds ---
790.0
